# VacationPy
----

In [1]:
# Dependencies and Setup:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key:
from api_keys import g_key

In [2]:
# Reading the 'csv file' with our weather data:
data = pd.read_csv('../OutputData/cities.csv')
data

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,Ozinki,51.18,49.68,70.79,76,54,8.66,RU,1594581213
1,Carballo,43.21,-8.69,71.47,50,0,10.29,ES,1594581144
2,Abu Samrah,35.30,37.18,74.91,69,0,14.61,SY,1594581213
3,Mataura,-46.19,168.86,39.00,83,88,3.00,NZ,1594580856
4,Hastings,50.86,0.57,63.45,51,98,5.39,GB,1594580894
...,...,...,...,...,...,...,...,...,...
556,Qinhuangdao,39.93,119.59,71.10,73,100,2.68,CN,1594581356
557,Solnechnyy,50.72,136.63,59.07,86,52,2.15,RU,1594581356
558,Sumenep,-7.02,113.87,78.57,82,100,6.31,ID,1594580909
559,Kushiro,42.98,144.37,53.46,92,28,4.43,JP,1594581357


In [3]:
# Pulling the information I will need to populate my ideal weather conditions dataframe:
lat_long = data[['Latitude', 'Longitude']]
lat_long

,Latitude,Longitude
0,51.18,49.68
1,43.21,-8.69
2,35.30,37.18
3,-46.19,168.86
4,50.86,0.57
...,...,...
556,39.93,119.59
557,50.72,136.63
558,-7.02,113.87
559,42.98,144.37


In [4]:
# Pulling the 'humidity' to add to the heatmap:
humidity = data['Humidity']
humidity

0      76
1      50
2      69
3      83
4      51
       ..
556    73
557    86
558    82
559    92
560    28
Name: Humidity, Length: 561, dtype: int64

In [5]:
# Generating the heatmap for the cities we have pulled:
gmaps.configure(api_key = g_key)
fig = gmaps.figure(zoom_level=1.5, center=(20,30))
heat_layer = gmaps.heatmap_layer(lat_long, weights=humidity, dissipating=False, max_intensity=10, point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Setting weather conditions to pull the cities with the best weather to find the vacation spots:
# I specifically set the winds to be less than 8 so I can populate no more than 10 cities/places:
ideal_weather = data.loc[(data['Max Temperature'] > 70) & 
                                 (data['Max Temperature'] < 80) & 
                                 (data['Wind Speed'] < 8) & 
                                 (data['Cloudiness'] == 0)]
ideal_weather

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
95,Nouadhibou,20.93,-17.03,78.80,65,0,6.93,MR,1594581235
135,Grand Gaube,-20.01,57.66,72.00,70,0,7.00,MU,1594581236
169,Annonay,45.24,4.67,77.79,31,0,6.93,FR,1594581253
174,Jinchang,38.50,102.17,70.27,49,0,4.68,CN,1594581255
179,Kiryat Gat,31.61,34.76,76.64,69,0,2.26,IL,1594580978
259,Anzio,41.49,12.62,78.31,81,0,1.99,IT,1594581275
299,Lozuvatka,48.06,33.29,73.40,46,0,6.71,UA,1594581285
447,Chornyanka,46.65,33.36,79.43,34,0,6.71,UA,1594581327
449,Malanje,-9.54,16.34,70.30,30,0,3.36,AO,1594581127
452,Sestri Levante,44.27,9.40,77.95,53,0,6.89,IT,1594581329


In [7]:
# Assuring there is 10 or less cities/places populating:
ideal_weather.count()

City               10
Latitude           10
Longitude          10
Max Temperature    10
Humidity           10
Cloudiness         10
Wind Speed         10
Country            10
Date               10
dtype: int64

In [8]:
# Using '.copy()' will help not interupt my existing 'ideal_weather' dataframe:
hotel_df = ideal_weather.copy()
hotel_df

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
95,Nouadhibou,20.93,-17.03,78.80,65,0,6.93,MR,1594581235
135,Grand Gaube,-20.01,57.66,72.00,70,0,7.00,MU,1594581236
169,Annonay,45.24,4.67,77.79,31,0,6.93,FR,1594581253
174,Jinchang,38.50,102.17,70.27,49,0,4.68,CN,1594581255
179,Kiryat Gat,31.61,34.76,76.64,69,0,2.26,IL,1594580978
259,Anzio,41.49,12.62,78.31,81,0,1.99,IT,1594581275
299,Lozuvatka,48.06,33.29,73.40,46,0,6.71,UA,1594581285
447,Chornyanka,46.65,33.36,79.43,34,0,6.71,UA,1594581327
449,Malanje,-9.54,16.34,70.30,30,0,3.36,AO,1594581127
452,Sestri Levante,44.27,9.40,77.95,53,0,6.89,IT,1594581329


In [9]:
# Adding a new column for 'Hotel Name':
hotel_df['Hotel Name'] = ""
hotel_df

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
95,Nouadhibou,20.93,-17.03,78.80,65,0,6.93,MR,1594581235,
135,Grand Gaube,-20.01,57.66,72.00,70,0,7.00,MU,1594581236,
169,Annonay,45.24,4.67,77.79,31,0,6.93,FR,1594581253,
174,Jinchang,38.50,102.17,70.27,49,0,4.68,CN,1594581255,
179,Kiryat Gat,31.61,34.76,76.64,69,0,2.26,IL,1594580978,
259,Anzio,41.49,12.62,78.31,81,0,1.99,IT,1594581275,
299,Lozuvatka,48.06,33.29,73.40,46,0,6.71,UA,1594581285,
447,Chornyanka,46.65,33.36,79.43,34,0,6.71,UA,1594581327,
449,Malanje,-9.54,16.34,70.30,30,0,3.36,AO,1594581127,
452,Sestri Levante,44.27,9.40,77.95,53,0,6.89,IT,1594581329,


In [10]:
# Searching for the closest hotels to the cities that meet the weather criteria:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

params = {
    "key": g_key,
    "radius": 5000,
    "type": "lodging",
    "keyword": "hotel"
}

for index, row in hotel_df.iterrows():    
    ideal_lat = row['Latitude']
    ideal_long = row['Longitude']
    params['location'] = f"{ideal_lat}, {ideal_long}"
    response = requests.get(url, params=params).json()
    results = response['results']   
# pprint(results)  

    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "-None available-"
        
        pass      

In [11]:
# Printing my 'hotel_df' to display the information pulled from the API Calls:
hotel_df

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
95,Nouadhibou,20.93,-17.03,78.80,65,0,6.93,MR,1594581235,Free Zone Hotel Nouadhibou
135,Grand Gaube,-20.01,57.66,72.00,70,0,7.00,MU,1594581236,Paradise Cove Boutique Hotel
169,Annonay,45.24,4.67,77.79,31,0,6.93,FR,1594581253,"Domaine de Saint Clair - Hôtel 4 étoiles, rest..."
174,Jinchang,38.50,102.17,70.27,49,0,4.68,CN,1594581255,Jinchang Hotel
179,Kiryat Gat,31.61,34.76,76.64,69,0,2.26,IL,1594580978,Desert Gat
259,Anzio,41.49,12.62,78.31,81,0,1.99,IT,1594581275,Astura Palace Hotel
299,Lozuvatka,48.06,33.29,73.40,46,0,6.71,UA,1594581285,Richka
447,Chornyanka,46.65,33.36,79.43,34,0,6.71,UA,1594581327,Zolotoy Fazan
449,Malanje,-9.54,16.34,70.30,30,0,3.36,AO,1594581127,Palanca Hotel
452,Sestri Levante,44.27,9.40,77.95,53,0,6.89,IT,1594581329,Hotel Helvetia


In [12]:
# NOTE: Do not change any of the code in this cell
# Using the template add the hotel marks to the heatmap

info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [13]:
# Add marker layer on top of heat map
# Display figure:

fig = gmaps.figure(zoom_level=1.5, center=(20,30))
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
heat_layer = gmaps.heatmap_layer(lat_long, weights=humidity, dissipating=False, max_intensity=10, point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))